# Install and import packages

In [1]:
#!pip install numpy==2.0 numba

In [1]:
#!pip install scipy

In [1]:
import numpy as np
from typing import List, Tuple
from dataclasses import dataclass
import time
from numba import jit
import timeit
import scipy

In [2]:
class OverwriteError(Exception):
    pass

In [3]:
class WindowSizeError(Exception):
    pass

# Forward and Backward for MLP networks

In [4]:
#@jit
def extend_input_by_one_vector(x_input: np.ndarray) -> np.ndarray:
    
    """

    """
    height, width = x_input.shape
    vector_extended = np.ones((height + 1, width))
    vector_extended[:height, :width] = x_input
    return vector_extended

In [5]:
#@jit
def pre_process_input(x_input: np.ndarray) -> np.ndarray:
    
    """

    """
    x_input = (x_input - np.mean(x_input)) / np.std(x_input)
    return x_input

In [6]:
#@jit
def apply_sigmoid(x_input: np.ndarray) -> np.ndarray:
    """

    """
    return 1 / (1 + np.exp(-x_input))

In [7]:
#@jit
def log_entropy(y_pred: np.ndarray, 
                y_label: np.ndarray) -> np.float16:
    """

    """
    return (-np.sum(y_label * np.log(y_pred) + (1 - y_label) * np.log(1 - y_pred))).astype(np.float16)

In [8]:
#@jit
def forward_pass(x_input: np.ndarray,
                weight_matrix_input_layer: np.ndarray,
                weight_matrix_first_hidden_layer: np.ndarray,
                weight_matrix_second_hidden_layer: np.ndarray,
                weight_matrix_output_layer: np.ndarray) -> Tuple[np.ndarray]:

    """

    """

    a1 = np.dot(weight_matrix_input_layer, x_input)
    h1 = apply_sigmoid(a1)
    h1 = extend_input_by_one_vector(h1)
    a2 = np.dot(weight_matrix_first_hidden_layer, h1)
    h2 = apply_sigmoid(a2)
    h2 = extend_input_by_one_vector(h2)
    a3 = np.dot(weight_matrix_second_hidden_layer, h2)
    h3 = apply_sigmoid(a3)
    h3 = extend_input_by_one_vector(h3)
    a4 = np.dot(weight_matrix_output_layer, h3)
    h4 = apply_sigmoid(a4)

    return (a1, h1, a2, h2, a3, h3, a4, h4)

In [9]:
learning_rate = 0.1
lambda_reg = 0.01

In [10]:
x_input = np.array([
                   [1,-2, -1],
                   [2 ,4 ,5],
                   [5,7,-4],
                   [-2,7,0],
                   [100, 20, -9]
                   ])
#print(x_input)

In [11]:
x_input = pre_process_input(x_input=x_input)

In [12]:
x_input = np.transpose(x_input)
x_input = extend_input_by_one_vector(x_input=x_input)

In [13]:
num_samples = x_input.shape[1]
num_iterations = 5000

In [14]:
y_label = np.transpose(np.array([
    [1, 0, 0], 
    [0, 1, 1], 
    [0, 0, 1], 
    [1, 1, 0],
    [1, 0, 0]
]))
#print(y_label)

In [15]:
input_dimension = 3
hidden_layer = [16, 16, 8]
output_dimension = 3

In [16]:
weight_matrix_input_layer = np.random.randn(hidden_layer[0], input_dimension + 1)
weight_matrix_first_hidden_layer = np.random.randn(hidden_layer[1], hidden_layer[0] + 1)
weight_matrix_second_hidden_layer = np.random.randn(hidden_layer[2], hidden_layer[1] + 1)
weight_matrix_output_layer = np.random.randn(output_dimension, hidden_layer[2] + 1)

In [17]:
#print(log_entropy(y_pred=h2[:,i], y_label=y_label[:,i]))

In [18]:
start_time = time.time()
for k in range(num_iterations):
    a1, h1, a2, h2, a3, h3, a4, h4 = forward_pass(x_input=x_input,
                                          weight_matrix_input_layer=weight_matrix_input_layer,
                                          weight_matrix_first_hidden_layer=weight_matrix_first_hidden_layer,
                                          weight_matrix_second_hidden_layer=weight_matrix_second_hidden_layer,
                                          weight_matrix_output_layer=weight_matrix_output_layer)
    update_weights_four = 0
    update_weights_three = 0
    update_weights_two = 0
    update_weights_one = 0
    for i in range(num_samples):
        
        backward_one = h4[:,i] - y_label[:,i]
        update_weights_four += np.stack([backward_one[j] * h3[:,i] for j in range(output_dimension)], axis=0).flatten()
        
        backward_two = np.dot(np.transpose(backward_one), weight_matrix_output_layer[:,:-1])
        backward_three = backward_two * (h3[:-1,i] * (1 - h3[:-1,i]))
        update_weights_three += np.stack([backward_three[j] * h2[:,i] for j in range(hidden_layer[2])], axis=0).flatten()
        
        backward_four = np.dot(np.transpose(backward_three), weight_matrix_second_hidden_layer[:,:-1])
        backward_five = backward_four * (h2[:-1,i] * (1 - h2[:-1,i]))
        update_weights_two += np.stack([backward_five[j] * h1[:,i] for j in range(hidden_layer[1])], axis=0).flatten()
        
        backward_six = np.dot(np.transpose(backward_five), weight_matrix_first_hidden_layer[:,:-1])
        backward_seven = backward_six * (h1[:-1,i] * (1 - h1[:-1,i]))
        update_weights_one += np.stack([backward_seven[j] * x_input[:,i] for j in range(hidden_layer[0])], axis=0).flatten()
    
    update_weights_one = -1/num_samples * update_weights_one - 2 * lambda_reg * np.sum(weight_matrix_input_layer.flatten())
    update_weights_two = -1/num_samples * update_weights_two - 2 * lambda_reg * np.sum(weight_matrix_first_hidden_layer.flatten())
    update_weights_three = -1/num_samples * update_weights_three - 2 * lambda_reg * np.sum(weight_matrix_second_hidden_layer.flatten())
    update_weights_four = -1/num_samples * update_weights_four - 2 * lambda_reg * np.sum(weight_matrix_output_layer.flatten())

    weight_matrix_output_layer = (weight_matrix_output_layer.flatten() \
                                  + learning_rate * update_weights_four).reshape(weight_matrix_output_layer.shape)
    weight_matrix_second_hidden_layer = (weight_matrix_second_hidden_layer.flatten() \
                                  + learning_rate * update_weights_three).reshape(weight_matrix_second_hidden_layer.shape)
    weight_matrix_first_hidden_layer = (weight_matrix_first_hidden_layer.flatten() \
                                  + learning_rate * update_weights_two).reshape(weight_matrix_first_hidden_layer.shape)
    weight_matrix_input_layer = (weight_matrix_input_layer.flatten() \
                                  + learning_rate * update_weights_one).reshape(weight_matrix_input_layer.shape)
end_time = time.time()
print(end_time - start_time)

12.079195261001587


In [19]:
print(h4)

[[9.98062679e-01 3.59900374e-03 4.81311450e-03 9.93920598e-01
  9.97596868e-01]
 [5.43001331e-03 9.93479407e-01 7.61350294e-03 9.91937684e-01
  4.29352539e-04]
 [1.53121084e-03 9.96068231e-01 9.95319796e-01 4.67335595e-03
  2.61778786e-03]]


# Forward and Backward for CNN networks

## Batch normalization

In [34]:
@jit
def batch_normalization_forward(array: np.ndarray) -> np.ndarray:
    
    """
    
    Substraction of mean and division by standard deviation.

    Args:
        array: The input array.
    Returns:
        Normalized vector.
    
    """
    return (array - np.mean(array)) / np.std(array)
    

    

In [21]:
array = np.random.randint(0, 255, 13 * 13 * 16)
print(array)

[186 253 247 ... 188  82 110]


In [22]:
start_time = time.time()
bnf = batch_normalization_forward(array)
end_time = time.time()
print(end_time - start_time)

2.318934679031372


In [23]:
@jit
def batch_normalization_backward_(array: np.ndarray) -> np.ndarray:
    """
    Subtraction of mean and division by standard deviation with faster computation.

    Args:
        array: The input array.
    Returns:
        Normalized vector.
    """
    array_length = array.shape[0]
    mean = np.mean(array)
    std = np.std(array)
    
    # Compute the first term (diagonal adjustment)
    diag_term = (np.eye(array_length) * array_length - 1) / (array_length * std)
    
    # Compute the second term (outer product-based)
    centered = array - mean
    outer_product_term = np.outer(centered, centered) / (array_length * std**3)
    
    # The final result
    derivative_normalization = diag_term - outer_product_term
    
    return derivative_normalization

In [24]:
@jit
def batch_normalization_backward(array: np.ndarray) -> np.ndarray:
    
    """
    
    Substraction of mean and division by standard deviation.

    Args:
        array: The input array.
    Returns:
        Normalized vector.
    
    """
    array_length = array.shape[0]
    mean = np.mean(array)
    std = np.std(array)
    derivative_normalization = np.zeros((array_length, array_length))

    for i in range(array_length):
        for j in range(i, array_length):
            derivative_normalization[i, j] = (int(i==j) * array_length - 1) / (array_length * std)\
            - (array[i] - mean) * (array[j] - mean) / (array_length * std * std * std)
            derivative_normalization[j, i] = derivative_normalization[i, j]
    
    return derivative_normalization

In [25]:
array = np.random.randint(0, 255, 13 * 13 * 16)

In [27]:
start_time = time.time()
bnb = batch_normalization_backward(array)
end_time = time.time()
print(end_time - start_time)

0.11031746864318848


## Max pooling

In [4]:
@jit
def max_pooling_forward(array: np.ndarray,
               window_size: Tuple[int]=(2,2)) -> Tuple[np.ndarray]:
    """

    Args:
        array:
        window_size:

    Returns:

    """
    row, col = array.shape
    row_pool, col_pool = row // 2, col // 2
    if row % window_size[0]:
        row_pool += 1
    if col % window_size[1]:
        col_pool += 1

    max_pooling_array_argmax = np.zeros((row_pool, col_pool), dtype=np.int64)
    max_pooling_array_max = np.zeros((row_pool, col_pool))
    max_pooling_array_argmax_global_indices = np.zeros((row_pool, col_pool), dtype=np.int64)
    for i in range(row_pool):
        for j in range(col_pool):
            max_pooling_array_argmax[i, j] = np.argmax(array[i*window_size[0]:(i+1)*window_size[0]\
                                                       ,j*window_size[1]:(j+1)*window_size[1]])
            max_pooling_array_max[i, j] = np.max(array[i*window_size[0]:(i+1)*window_size[0]\
                                                       ,j*window_size[1]:(j+1)*window_size[1]])

            max_pooling_array_argmax_global_indices[i, j] = ((2 * i \
                                                              + max_pooling_array_argmax[i,j] // 2) \
                                                             * col + 2 * j + max_pooling_array_argmax[i,j] % 2)

    return max_pooling_array_argmax_global_indices.flatten(), max_pooling_array_max


In [5]:
@jit
def max_pooling_backward(array: np.ndarray,
                        array_max_indices: np.ndarray) -> np.ndarray:
    """

    Args:
        array:
        array_max_indiices:

    Returns:

    """
    row, col = array.shape
    len_indices = len(array_max_indices)
    result = np.zeros((len_indices, row * col))
    for i in range(len_indices):
        for j in range(row * col):
            result[i,j] = int(array_max_indices[i] == j)
        
    return result

In [6]:
array = np.random.randint(0, 255, (8, 28, 28, 1))
#print(array[:,:,0,0])

In [7]:
channel, row, col, batch_size = array.shape

In [8]:
start_time = time.time()
res = [[max_pooling_forward(array[i, : , :, j]) for i in range(channel)] for j in range(batch_size)]
end_time = time.time()
print(end_time - start_time)

3.845829486846924


In [9]:
start_time = time.time()
mpb = max_pooling_backward(array[0,:,:,0], res[0][0][0])
end_time = time.time()
print(end_time - start_time)

0.4724555015563965


## Alternative max pooling

In [18]:
def max_pooling_forward(array: np.ndarray,
               window_size: int=2) -> List[np.ndarray]:
    """

    Args:
        array:
        window_size:

    Returns:

    """

    row, col = array.shape
    if row % window_size:
        #array = np.vstack((array, np.array([array[-1,:]])))
        array = np.vstack((array, -np.inf * np.ones((1, col))))
        
    if col % window_size:
        row, col = array.shape
        #array = np.hstack((array, np.transpose(np.array([array[:,-1]]))))
        array = np.hstack((array, -np.inf * np.ones((row, 1))))

    row, col = array.shape
    row_output = row // window_size
    col_output = col // window_size
    array_window_reshaped = array[:row_output*window_size, :col_output*window_size] \
    .reshape(row_output, window_size, col_output, window_size).swapaxes(1,2) \
    .reshape(-1, window_size ** 2)

    array_window_reshaped_argmax = array_window_reshaped.argmax(axis=1).reshape(row_output, col_output)
    array_window_reshaped_max = array_window_reshaped.max(axis=1).reshape(row_output, col_output)

    #max_pooling_array_argmax_global_indices \
    #            = np.array([[((2 * i + int(array_window_reshaped_argmax[i,j] // 2)) * col + 2 * j + array_window_reshaped_argmax[i,j] % 2) \
    #           for j in range(col_output)] \
    #          for i in range(row_output)]).flatten()

    return array_window_reshaped_max#, max_pooling_array_argmax_global_indices

In [19]:
array = np.random.randint(0, 255, (1001, 1001))

In [20]:
array

array([[ 23,  23,  58, ...,  68, 165, 250],
       [ 71, 172,  32, ..., 123, 184, 153],
       [173, 146,  83, ..., 112,   1, 206],
       ...,
       [197, 231, 133, ..., 117, 223,  85],
       [147,  37,   4, ..., 119,  73,  59],
       [ 10, 220, 140, ..., 227, 105, 237]])

In [21]:
start_time = time.time()
x = max_pooling_forward(array)
end_time = time.time()
print(end_time - start_time)
print(x)

0.06948995590209961
[[172. 165. 170. ... 215. 184. 250.]
 [218. 207. 224. ... 247. 112. 206.]
 [219. 252. 161. ... 180. 163. 253.]
 ...
 [214. 191.  81. ... 251. 171. 215.]
 [231. 228. 193. ... 175. 223.  85.]
 [220. 140. 188. ...  78. 227. 237.]]


In [22]:
max_pooling_forward(mat)

NameError: name 'mat' is not defined

In [ ]:
max_pooling_forward(mat)

In [ ]:
max_pooling_forward(mat)[[0, 1],[1, 2]]

In [ ]:
mat = np.random.randint(0, 10, (5, 5))

## Convolution

In [10]:
array = np.random.randint(0, 255, (6, 6))
#print(array)
filter_convolution = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
filter_convolution = np.random.randn(3,3)
offset = np.random.rand(1)[0]
print(offset)
print(filter_convolution)

0.7750820433310169
[[ 0.44629861  1.78712402 -0.50723247]
 [ 0.09509415 -1.62199944 -0.98359166]
 [ 0.93698612  1.05682671  0.60621636]]


In [11]:
array_shape = array.shape
filter_shape = filter_convolution.shape

In [12]:
@jit
def create_base_conv_vector(array_shape: Tuple[int], 
                              filter_conv: np.ndarray) -> np.ndarray:
    """
    Args:
        array: The shape of the matrix on which
               we want to apply a  convolution.
        filter_conv: The filter matrix. 
    Returns:
        The basic convolutional vector from which we will 
        construct the convolution matrix.
    
    """
    filter_conv_row, filter_conv_col = filter_conv.shape
    conv_vector = np.zeros(array_shape[0] * array_shape[1])
    for i in range(filter_conv_row):
        conv_vector[array_shape[1]*i:array_shape[1]*i+filter_conv_row] = filter_conv[i,:]
    return conv_vector


In [13]:
@jit
def create_convolution_matrix(array_shape: Tuple[int], 
                              filter_conv: np.ndarray) -> np.ndarray:
    """
    Args:
        array: The shape of the matrix on which
               we want to apply a  convolution.
        filter_conv: The filter matrix. 
    Returns:
        The expanded filter matrix.
    
    """
    filter_conv_row, filter_conv_col = filter_conv.shape
    matrix_conv_rep = np.zeros(((array_shape[0] - filter_conv_row + 1) * (array_shape[1] - filter_conv_col + 1) \
                                , array_shape[0] * array_shape[1]))
    base_conv_vector = create_base_conv_vector(array_shape, filter_conv)
    for i in range((array_shape[1] - filter_conv_col + 1) * (array_shape[0] - filter_conv_row + 1)):
        matrix_conv_rep[i,:] = np.roll(base_conv_vector, int(i // (array_shape[0] - filter_conv_row + 1)) * array_shape[1] \
        + i % (array_shape[0] - filter_conv_row + 1))

    return matrix_conv_rep

            

In [14]:
start_time = time.time()
convolution_matrix = create_convolution_matrix(array.shape, filter_convolution)
end_time = time.time()
print(end_time - start_time)

1.8901357650756836


In [15]:
def convolution_forward(array: np.ndarray,
                       conv_matrix: np.ndarray) -> np.ndarray:
    """

    """
    return np.dot(conv_matrix, array)

In [16]:
start_time = time.time()
convolution_matrix = create_convolution_matrix(array.shape, filter_convolution)
conv_forward = convolution_forward(array.flatten(), convolution_matrix) \
    .reshape((array_shape[0] - filter_shape[0] + 1, array_shape[1] - filter_shape[1] + 1)) \
    + offset
max_pool_forward = max_pooling_forward(conv_forward)
end_time = time.time()
print(end_time - start_time)

1.9109060764312744


In [17]:
@jit
def convolution_backward_update(array: np.ndarray,
                        filter_conv_shape: Tuple[int] = (3, 3)) -> np.ndarray:
    """

    """

    array_shape = array.shape
    filter_conv_update_matrix = np.zeros(((array_shape[0] - filter_conv_shape[0] + 1) \
                                   * (array_shape[1] - filter_conv_shape[1] + 1) \
                                , filter_conv_shape[0] * filter_conv_shape[1]))
    index = 0
    for i in range(array_shape[0] - filter_conv_shape[0] + 1):
        for j in range(array_shape[1] - filter_conv_shape[1] + 1):
            filter_conv_update_matrix[index] = array[i:(i+3), j:(j+3)].flatten()
            index += 1
    

    return filter_conv_update_matrix

In [18]:
start_time = time.time()
filter_conv_shape = filter_convolution.shape
conv_backward = convolution_backward_update(array, 
                    filter_conv_shape)
end_time = time.time()
print(end_time - start_time)

0.8693954944610596


## Backpropagation_update_matrix

In [19]:
@jit
def backpropagation_update_matrix(array: np.ndarray,
                                 num_filters_old: int,
                                 num_filters_new: int,
                                 array_conv_shape: int) -> np.ndarray:
    """

    """
    filter_conv_update_matrix = np.zeros((array_conv_shape * array_conv_shape * num_filters_new, \
                                         (filter_col * filter_row) * num_filters_old * num_filters_new + num_filters_new))
    index = 0
    for k in range(num_filters_old):
        array_sliced = array[k, :,:]
        local_update_matrix = convolution_backward_update(array=array_sliced)
        for l in range(num_filters_new):
            filter_conv_update_matrix[l * array_conv_shape*array_conv_shape:(l+1) * array_conv_shape * array_conv_shape,\
            (num_filters_new * k + l) * filter_col * filter_row\
            :(num_filters_new * k + l + 1) * filter_col * filter_row] \
            = local_update_matrix

    for k in range(num_filters_new-1):
        filter_conv_update_matrix[k * array_conv_shape * array_conv_shape:(k + 1) * array_conv_shape * array_conv_shape\
        ,-(num_filters_new-k):-(num_filters_new-k-1)] = 1

    k = num_filters_new - 1
    filter_conv_update_matrix[k * array_conv_shape * array_conv_shape:(k + 1) * array_conv_shape * array_conv_shape\
        ,-1:] = 1
    
    return 1 / num_filters_old * filter_conv_update_matrix

In [20]:
num_filters_old = 4
num_filters_new = 3
filter_row, filter_col = [3, 3]

In [21]:
array_shape = (4, 4)

In [22]:
array_conv_shape = array_shape[0] - 3 + 1

In [23]:
array = np.random.randint(0, 255, (num_filters_old, array_shape[0], array_shape[1]))
filter_convolution = np.random.randn(num_filters_new, num_filters_old, filter_row, filter_col)
offset = np.random.rand(num_filters_new, 1).flatten()
#print(offset)
#print(filter_convolution)

In [24]:
filter_convolution.shape

(3, 4, 3, 3)

In [25]:
start_time = time.time()
update_matrix = backpropagation_update_matrix(array=array,
                             num_filters_old=num_filters_old,
                             num_filters_new=num_filters_new,
                             array_conv_shape=array_conv_shape)
end_time = time.time()
print(end_time - start_time)

1.8293671607971191


In [26]:
#update_matrix[-4:,:]

## Full network

### Specify input and parameters

In [27]:
@jit
def normalize_input_image(array: np.ndarray) -> np.ndarray:
    """
    We normalize the input image tensor by dividing by 255.
    """
    return 1 / 255 * array 

In [49]:
@jit
def create_convolution_matrix_stacked(filter_convolution_sliced: np.ndarray,
                                     row_array: int,
                                     col_array: int) -> np.ndarray:
    """

    """
    num_filter, row_filter, col_filter = filter_convolution_sliced.shape
    row_res, col_res = (row_array - row_filter + 1, col_array - col_filter + 1)
    dimesion_row, dimension_col = (row_res * col_res, row_array * col_array)
    convolution_matrix_stacked = np.zeros((dimesion_row, dimension_col * num_filter))
    for j in range(num_filter):
        convolution_matrix_stacked[:, dimension_col * j: dimension_col * (j+1)]\
        = create_convolution_matrix((row_array, col_array), filter_convolution_sliced[j,:,:])

    return convolution_matrix_stacked

In [50]:
@jit
def forward_backward_step_convolution_module(filter_convolution: np.array,
                                            offset: np.ndarray,
                                            array: np.array,
                                            num_filters_old: int,
                                            num_filters_new: int) -> Tuple[np.ndarray]:
    """

    """
    num_filter_old, row_array, col_array = array.shape
    row_conv, col_conv = [row_array - filter_conv_row + 1, col_array - filter_conv_col + 1]
    row_pool, col_pool = [row_conv // 2, col_conv // 2]
    matrix_convolution_max_pooling_backward = np.zeros((row_pool ** 2 * num_filters_new, row_array ** 2 * num_filter_old))
    feature_map_final_flatten = np.zeros(row_pool ** 2 * num_filters_new)
    max_pooling_backward_matrix = np.zeros((row_pool ** 2 * num_filters_new, row_conv ** 2 * num_filters_new))
    for j in range(num_filters_new):
        filter_convolution_sliced = filter_convolution[j,:,:,:]
        convolution_matrix_stacked = create_convolution_matrix_stacked(filter_convolution_sliced=filter_convolution_sliced,
                                                                row_array=row_array,
                                                               col_array=col_array)
        feature_vector_stacked = array.flatten()
        feature_map = 1 / num_filters_old * np.dot(convolution_matrix_stacked, feature_vector_stacked) + offset[j]
        feature_map_reshaped = feature_map.reshape(row_conv, col_conv)
        feature_map_max_pooling_indices, feature_map_max_pooling = max_pooling_forward(array=feature_map_reshaped)
        feature_map_max_pooling_flatten = feature_map_max_pooling.flatten()
        feature_map_final_flatten[row_pool ** 2 * j:row_pool ** 2 * (j+1)] = feature_map_max_pooling_flatten
        mpb = max_pooling_backward(array=feature_map_reshaped, array_max_indices=feature_map_max_pooling_indices)
        max_pooling_backward_matrix[row_pool ** 2 * j:row_pool ** 2 * (j+1), row_conv ** 2 * j:row_conv ** 2 * (j+1)] = mpb
        matrix_convolution_max_pooling_backward[row_pool ** 2 * j:row_pool ** 2 * (j+1),:] = 1 / num_filters_old * np.dot(mpb, convolution_matrix_stacked)
    bnb = batch_normalization_backward(feature_map_final_flatten) 
    array_batch_normalization = batch_normalization_forward(feature_map_final_flatten).reshape((num_filters_new, row_pool, col_pool))
    norm_backpropagation = np.dot(bnb, matrix_convolution_max_pooling_backward)
    norm_max_pooling_backpropagation = np.dot(bnb, max_pooling_backward_matrix)
    return array_batch_normalization, norm_backpropagation, norm_max_pooling_backpropagation  

### Example cnn

In [132]:
num_filters = [1, 8, 4]
num_iterations = 1
learning_rate = 0.1
lambda_reg = 0.01
num_samples = 1
y_label = np.transpose(np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
]))
print(y_label)

In [137]:
stage = 0
input_size = (num_filters[stage], 30, 30)
array = np.random.randint(0, 255, input_size)
array = normalize_input_image(array=array)
filter_conv_row, filter_conv_col = (3, 3)
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]
filter_convolution_one = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_one = np.random.rand(num_filters_new, 1).flatten()
stage = 1
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]
filter_convolution_two = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_two = np.random.rand(num_filters_new, 1).flatten()
input_dimension = 144
hidden_layer = [8]
output_dimension = 10

In [140]:
weight_matrix_input_layer = np.random.randn(hidden_layer[0], input_dimension + 1)
weight_matrix_output_layer = np.random.randn(output_dimension, hidden_layer[0] + 1)

### First convolution, max_pooling and batch normalization

In [ ]:
for i in range(30):
    stage = 0
    input_size = (num_filters[stage], 30, 30)
    filter_conv_row, filter_conv_col = (3, 3)
    num_filters_old = num_filters[stage]
    num_filters_new = num_filters[stage+1]
    num_filter_old, row_array, col_array = array.shape
    row_conv, col_conv = [row_array - filter_conv_row + 1, col_array - filter_conv_col + 1]
    row_pool, col_pool = [row_conv // 2, col_conv // 2]
    array_one, derivative_one, norm_max_pooling_backward_one = forward_backward_step_convolution_module(filter_convolution=filter_convolution_one,
                                                                                                        offset=offset_one,
                                            array=array,
                                            num_filters_old=num_filter_old,
                                            num_filters_new=num_filters_new)
    update_matrix_conv_one = backpropagation_update_matrix(array=array,
                                 num_filters_old=num_filter_old,
                                 num_filters_new=num_filters_new,
                                 array_conv_shape=row_conv)
    stage = 1
    num_filters_old = num_filters[stage]
    num_filters_new = num_filters[stage+1]
    print(num_filter_old, num_filters_new)
    num_filter_old, row_array, col_array = array_one.shape
    row_conv, col_conv = [row_array - filter_conv_row + 1, col_array - filter_conv_col + 1]
    row_pool, col_pool = [row_conv // 2, col_conv // 2]
    array_two, derivative_two, norm_max_pooling_backward_two = forward_backward_step_convolution_module(filter_convolution=filter_convolution_two,
                                                                                                        offset=offset_two,
                                            array=array_one,
                                            num_filters_old=num_filter_old,
                                            num_filters_new=num_filters_new)
    update_matrix_conv_two = backpropagation_update_matrix(array=array_one,
                                 num_filters_old=num_filter_old,
                                 num_filters_new=num_filters_new,
                                 array_conv_shape=row_conv)
    array_three = array_two.flatten()
    array_three = np.expand_dims(array_three, axis=0)
    input_dimension = array_three.shape[1]
    print("----------------MLP-----------------")
    hidden_layer = [8]
    output_dimension = 10
    x_input = np.transpose(array_three)
    x_input = extend_input_by_one_vector(x_input=x_input)
    k = 0
    a1, h1, a2, h2 = forward_pass(x_input=x_input,
                                              weight_matrix_input_layer=weight_matrix_input_layer,
                                              weight_matrix_output_layer=weight_matrix_output_layer)
    print(h2)
    update_weights_two = 0
    update_weights_one = 0
    i = 0
    backward_one = h2[:,i] - y_label[:,i]
    update_weights_two += np.stack([backward_one[j] * h1[:,i] for j in range(output_dimension)], axis=0).flatten()
    
    update_weights_two = -1/1 * update_weights_two
    
    backward_two = np.dot(np.transpose(backward_one), weight_matrix_output_layer[:,:-1])
    backward_three = backward_two * (h1[:-1,i] * (1 - h1[:-1,i]))
    update_weights_one += np.stack([backward_three[j] * x_input[:,i] for j in range(hidden_layer[0])], axis=0).flatten()
    
    update_weights_one = -1/1 * update_weights_one
    
    backward_four = np.dot(np.transpose(backward_three), weight_matrix_input_layer[:,:-1])
    weight_matrix_output_layer = (weight_matrix_output_layer.flatten() \
                                      + learning_rate * update_weights_two).reshape(weight_matrix_output_layer.shape)
    weight_matrix_input_layer = (weight_matrix_input_layer.flatten() \
                                      + learning_rate * update_weights_one).reshape(weight_matrix_input_layer.shape)
    
    
    backpropagation_second_norm_max_pooling = np.dot(backward_four, norm_max_pooling_backward_two)
    update_second_conv_weights = np.dot(backpropagation_second_norm_max_pooling, update_matrix_conv_two)
    update_second_conv_weights = -1/1 * update_second_conv_weights
    update_second_conv_weights_reshaped = update_second_conv_weights[:-num_filters[2]].reshape(num_filters[2], num_filters[1], 3, 3)
    filter_convolution_two = filter_convolution_two + learning_rate * update_second_conv_weights_reshaped
    offset_two = offset_two + learning_rate * update_second_conv_weights[-num_filters[2]:]
    
    
    backpropagation_first_norm_max_pooling = np.dot(np.dot(backward_four, derivative_two), norm_max_pooling_backward_one)
    update_first_conv_weights = np.dot(backpropagation_first_norm_max_pooling, update_matrix_conv_one)
    update_first_conv_weights = -1/1 * update_first_conv_weights
    update_first_conv_weights_reshaped = update_first_conv_weights[:-num_filters[1]].reshape(num_filters[1], num_filters[0], 3, 3)
    filter_convolution_one = filter_convolution_one + learning_rate * update_first_conv_weights_reshaped
    offset_one = offset_one + learning_rate * update_first_conv_weights[-num_filters[1]:]

In [105]:
stage = 0

In [106]:
input_size = (num_filters[stage], 30, 30)

In [107]:
array = np.random.randint(0, 255, input_size)
array = normalize_input_image(array=array)

In [108]:
filter_conv_row, filter_conv_col = (3, 3)

In [109]:
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]

In [110]:
filter_convolution_one = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_one = np.random.rand(num_filters_new, 1).flatten()

In [111]:
num_filter_old, row_array, col_array = array.shape
row_conv, col_conv = [row_array - filter_conv_row + 1, col_array - filter_conv_col + 1]
row_pool, col_pool = [row_conv // 2, col_conv // 2]

In [112]:
start_time = time.time()
array_one, derivative_one, norm_max_pooling_backward_one = forward_backward_step_convolution_module(filter_convolution=filter_convolution_one,
                                                                                                    offset=offset_one,
                                        array=array,
                                        num_filters_old=num_filter_old,
                                        num_filters_new=num_filters_new)
end_time = time.time()
print(end_time - start_time)

4.48169469833374


In [113]:
update_matrix_conv_one = backpropagation_update_matrix(array=array,
                             num_filters_old=num_filter_old,
                             num_filters_new=num_filters_new,
                             array_conv_shape=row_conv)

In [114]:
array_one.shape

(8, 14, 14)

### Second convolution, max_pooling and batch normalization

In [115]:
stage = 1

In [116]:
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]
print(num_filter_old, num_filters_new)

1 4


In [117]:
filter_convolution_two = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_two = np.random.rand(num_filters_new, 1).flatten()

In [118]:
num_filter_old, row_array, col_array = array_one.shape
row_conv, col_conv = [row_array - filter_conv_row + 1, col_array - filter_conv_col + 1]
row_pool, col_pool = [row_conv // 2, col_conv // 2]

In [119]:
start_time = time.time()
array_two, derivative_two, norm_max_pooling_backward_two = forward_backward_step_convolution_module(filter_convolution=filter_convolution_two,
                                                                                                    offset=offset_two,
                                        array=array_one,
                                        num_filters_old=num_filter_old,
                                        num_filters_new=num_filters_new)
end_time = time.time()
print(end_time - start_time)

0.08735275268554688


In [120]:
update_matrix_conv_two = backpropagation_update_matrix(array=array_one,
                             num_filters_old=num_filter_old,
                             num_filters_new=num_filters_new,
                             array_conv_shape=row_conv)

In [121]:
array_three = array_two.flatten()
array_three = np.expand_dims(array_three, axis=0)

In [122]:
array_three.shape[1]

144

### MLP

In [98]:
@jit
def extend_input_by_one_vector(x_input: np.ndarray) -> np.ndarray:
    
    """

    """
    height, width = x_input.shape
    vector_extended = np.ones((height + 1, width))
    vector_extended[:height, :width] = x_input
    return vector_extended

In [99]:
@jit
def apply_sigmoid(x_input: np.ndarray) -> np.ndarray:
    """

    """
    return 1 / (1 + np.exp(-x_input))

In [100]:
@jit
def forward_pass(x_input: np.ndarray,
                weight_matrix_input_layer: np.ndarray,
                weight_matrix_output_layer: np.ndarray) -> Tuple[np.ndarray]:

    """

    """

    a1 = np.dot(weight_matrix_input_layer, x_input)
    h1 = apply_sigmoid(a1)
    h1 = extend_input_by_one_vector(h1)
    a2 = np.dot(weight_matrix_output_layer, h1)
    h2 = apply_sigmoid(a2)

    return (a1, h1, a2, h2)

In [101]:
input_dimension = array_three.shape[1]
hidden_layer = [8]
output_dimension = 10

NameError: name 'array_three' is not defined

In [127]:
x_input = np.transpose(array_three)
x_input = extend_input_by_one_vector(x_input=x_input)

In [128]:
#weight_matrix_input_layer = np.random.randn(hidden_layer[0], input_dimension + 1)
#weight_matrix_output_layer = np.random.randn(output_dimension, hidden_layer[0] + 1)

In [129]:
k = 0

In [130]:
a1, h1, a2, h2 = forward_pass(x_input=x_input,
                                          weight_matrix_input_layer=weight_matrix_input_layer,
                                          weight_matrix_output_layer=weight_matrix_output_layer)

In [131]:
h2

array([[0.0987318 ],
       [0.5672034 ],
       [0.00873734],
       [0.94498739],
       [0.87578339],
       [0.12611772],
       [0.28211042],
       [0.22324196],
       [0.61287511],
       [0.05421787]])

#### Backpropagation

In [382]:
update_weights_two = 0
update_weights_one = 0
i = 0
backward_one = h2[:,i] - y_label[:,i]
update_weights_two += np.stack([backward_one[j] * h1[:,i] for j in range(output_dimension)], axis=0).flatten()

backward_two = np.dot(np.transpose(backward_one), weight_matrix_output_layer[:,:-1])
backward_three = backward_two * (h1[:-1,i] * (1 - h1[:-1,i]))
update_weights_one += np.stack([backward_three[j] * x_input[:,i] for j in range(hidden_layer[0])], axis=0).flatten()

backward_four = np.dot(np.transpose(backward_three), weight_matrix_input_layer[:,:-1])
weight_matrix_output_layer = (weight_matrix_output_layer.flatten() \
                                  + learning_rate * update_weights_two).reshape(weight_matrix_output_layer.shape)
weight_matrix_input_layer = (weight_matrix_input_layer.flatten() \
                                  + learning_rate * update_weights_one).reshape(weight_matrix_input_layer.shape)

In [385]:
#backward_four.shape

In [386]:
backpropagation_second_norm_max_pooling = np.dot(backward_four, norm_max_pooling_backward_two)
update_second_conv_weights = np.dot(backpropagation_second_norm_max_pooling, update_matrix_conv_two)
update_second_conv_weights_reshaped = update_second_conv_weights[:-num_filters[2]].reshape(num_filters[2], num_filters[1], 3, 3)
filter_convolution_two = filter_convolution_two + learning_rate * update_second_conv_weights_reshaped
offset_two = offset_two + learning_rate * update_second_conv_weights[-num_filters[2]:]

In [389]:
#print(filter_convolution_two.shape)
#print(update_second_conv_weights[:-num_filters[2]].reshape(num_filters[2], num_filters[1], 3, 3).shape)

In [390]:
#offset_two

In [391]:
#update_second_conv_weights[-num_filters[2]:]

In [392]:
#print(filter_convolution_two.shape)
#print(offset_two.shape)

In [393]:
#filter_convolution_two = filter_convolution_two + learning_rate * update_second_conv_weights_reshaped
#offset_two = offset_two + learning_rate * update_second_conv_weights[-num_filters[2]:]

In [394]:
#print(filter_convolution_two.shape)
#print(offset_two.shape)

In [395]:
assert update_second_conv_weights.shape[0] == num_filters[2] * num_filters[1] * 9 + num_filters[2]

In [396]:
backpropagation_first_norm_max_pooling = np.dot(np.dot(backward_four, derivative_two), norm_max_pooling_backward_one)
update_first_conv_weights = np.dot(backpropagation_first_norm_max_pooling, update_matrix_conv_one)
update_first_conv_weights_reshaped = update_first_conv_weights[:-num_filters[1]].reshape(num_filters[1], num_filters[0], 3, 3)
filter_convolution_one = filter_convolution_one + learning_rate * update_first_conv_weights_reshaped
offset_one = offset_one + learning_rate * update_first_conv_weights[-num_filters[1]:]

In [398]:
assert update_first_conv_weights.shape[0] == num_filters[1] * num_filters[0] * 9 + num_filters[1]

In [399]:
#update_first_conv_weights_reshaped = update_first_conv_weights[:-num_filters[1]].reshape(num_filters[1], num_filters[0], 3, 3)

In [400]:
#print(filter_convolution_one.shape)
#print(offset_one.shape)

In [401]:
#filter_convolution_one = filter_convolution_one + learning_rate * update_first_conv_weights_reshaped
#offset_one = offset_one + learning_rate * update_first_conv_weights[-num_filters[1]:]

In [402]:
#print(filter_convolution_one.shape)
#print(offset_one.shape)

In [403]:
#update_first_conv_weights[:-num_filters[1]].reshape(num_filters[0], num_filters[1], 3, 3).swapaxes(0, 1)

# Make it clean

## Batch normalization

### Forward

In [28]:
@jit
def batch_normalization_forward(array: np.ndarray) -> np.ndarray:
    
    """
    
    Substraction of mean and division by standard deviation.

    Args:
        array: The input array.
    Returns:
        Normalized vector.
    
    """
    return (array - np.mean(array)) / np.std(array)

In [29]:
array = np.random.randint(0, 255, 13 * 13 * 16)
print(array)
start_time = time.time()
bnf = batch_normalization_forward(array)
end_time = time.time()
print(end_time - start_time)

[  7  63 130 ... 117  46 220]
1.3185274600982666


### Backward

In [30]:
@jit
def batch_normalization_backward(array: np.ndarray) -> np.ndarray:
    
    """
    
    Substraction of mean and division by standard deviation.

    Args:
        array: The input array.
    Returns:
        Normalized vector.
    
    """
    array_length = array.shape[0]
    mean = np.mean(array)
    std = np.std(array)
    derivative_normalization = np.zeros((array_length, array_length))

    for i in range(array_length):
        for j in range(i, array_length):
            derivative_normalization[i, j] = (int(i==j) * array_length - 1) / (array_length * std)\
            - (array[i] - mean) * (array[j] - mean) / (array_length * std * std * std)
            derivative_normalization[j, i] = derivative_normalization[i, j]
    
    return derivative_normalization

In [31]:
array = np.random.randint(0, 255, 13 * 13 * 16)
start_time = time.time()
bnb = batch_normalization_backward(array)
end_time = time.time()
print(end_time - start_time)

0.5954625606536865


## Max pooling

### Forward

In [32]:
@jit
def max_pooling_forward(array: np.ndarray,
               window_size: Tuple[int]=(2,2)) -> Tuple[np.ndarray]:
    """

    Args:
        array:
        window_size:

    Returns:

    """
    channel, row, col = array.shape
    row_pool, col_pool = row // 2, col // 2
    if row % window_size[0]:
        row_pool += 1
    if col % window_size[1]:
        col_pool += 1

    max_pooling_array_argmax = np.zeros((channel, row_pool, col_pool), dtype=np.int64)
    max_pooling_array_max = np.zeros((channel, row_pool, col_pool))
    max_pooling_array_argmax_global_indices = np.zeros((channel, row_pool, col_pool), dtype=np.int64)
    for c in range(channel):
        for i in range(row_pool):
            for j in range(col_pool):
                max_pooling_array_argmax[c, i, j] = np.argmax(array[c, i*window_size[0]:(i+1)*window_size[0]\
                                                           ,j*window_size[1]:(j+1)*window_size[1]])
                max_pooling_array_max[c, i, j] = np.max(array[c, i*window_size[0]:(i+1)*window_size[0]\
                                                           ,j*window_size[1]:(j+1)*window_size[1]])
    
                max_pooling_array_argmax_global_indices[c, i, j] = ((2 * i \
                                                                  + max_pooling_array_argmax[c, i,j] // 2) \
                                                                 * col + 2 * j + max_pooling_array_argmax[c, i,j] % 2)

    return max_pooling_array_argmax_global_indices.reshape(channel, row_pool * row_pool), max_pooling_array_max


In [33]:
array = np.random.randint(0, 255, (32, 28, 28))

In [34]:
array.shape

(32, 28, 28)

In [35]:
start_time = time.time()
a, b = max_pooling_forward(array=array)
end_time = time.time()
print(end_time - start_time)

1.1962323188781738


### Backward

In [36]:
@jit
def max_pooling_backward(array_shape: Tuple[int],
                        array_max_indices: np.ndarray) -> np.ndarray:
    """

    Args:
        array:
        array_max_indices:

    Returns:

    """
    channel, row, col = array_shape
    size_feature_map = row * col
    _, num_max_indices = array_max_indices.shape

    array_backpropagation = np.zeros((channel * num_max_indices, channel * size_feature_map))

    for c in range(channel):
        array_max_indices_sliced = array_max_indices[c,:]
        for i in range(num_max_indices):
            for j in range(row * col):
                array_backpropagation[c * num_max_indices  + i, c * size_feature_map + j] = int(array_max_indices_sliced[i] == j)

    return array_backpropagation

In [37]:
array = np.random.randint(0, 255, (2, 4, 4))

In [38]:
start_time = time.time()
max_indices, pooling_array = max_pooling_forward(array=array)
end_time = time.time()
print(end_time - start_time)

0.00014281272888183594


In [39]:
start_time = time.time()
mpb = max_pooling_backward(array.shape, max_indices)
end_time = time.time()
print(end_time - start_time)

0.42421555519104004


## Convolution

### Forward

In [40]:
@jit
def create_base_conv_vector(array_shape: Tuple[int], 
                              filter_convolution: np.ndarray) -> np.ndarray:
    """
    Args:
        array: The shape of the matrix on which
               we want to apply a  convolution.
        filter_conv: The filter matrix. 
    Returns:
        The basic convolutional vector from which we will 
        construct the convolution matrix.
    
    """
    filter_conv_row, filter_conv_col = filter_convolution.shape
    conv_vector = np.zeros(array_shape[0] * array_shape[1])
    for i in range(filter_conv_row):
        conv_vector[array_shape[1]*i:array_shape[1]*i+filter_conv_row] = filter_convolution[i,:]
    return conv_vector


In [41]:
@jit
def create_convolution_matrix(array_shape: Tuple[int], 
                              filter_convolution: np.ndarray,
                              offset: np.array) -> np.ndarray:
    """
    Args:
        array: The shape of the matrix on which
               we want to apply a  convolution.
        filter_conv: The filter matrix. 
    Returns:
        The expanded filter matrix.
    
    """

    _, row, col = array_shape
    size_old_feature_map = row * col
    num_filter_new, num_filter_old, filter_conv_row, filter_conv_col = filter_convolution.shape
    feature_map_conv_row_col = row - filter_conv_row + 1
    size_new_feature_map = feature_map_conv_row_col * feature_map_conv_row_col
    
    matrix_conv_rep = np.zeros((size_new_feature_map * num_filter_new \
                                , size_old_feature_map * num_filter_old + 1))
    
    for i in range(num_filter_new):
        for j in range(num_filter_old):
            base_conv_vector = create_base_conv_vector((row, col), filter_convolution[i, j])
            for k in range(size_new_feature_map):
                matrix_conv_rep[i * size_new_feature_map + k, j * size_old_feature_map:(j+1) * size_old_feature_map] \
                = np.roll(base_conv_vector, int(k // (feature_map_conv_row_col)) * col \
                + k % feature_map_conv_row_col)
        matrix_conv_rep[i * size_new_feature_map:(i+1) * size_new_feature_map,-1] = offset[i]
    return matrix_conv_rep            

In [42]:
@jit
def convolution_forward(array: np.ndarray,
                        conv_matrix: np.ndarray) -> np.ndarray:
    """

    """
    array = array.astype(np.float32)
    conv_matrix = conv_matrix.astype(np.float32)
    return np.dot(conv_matrix, array)

In [43]:
array = np.random.randint(0, 255, (1, 30, 30))
#print(array)
num_filter_new, num_filter_old = (16 ,1)
filter_convolution = np.random.randn(num_filter_new, num_filter_old, 3, 3)
offset = np.random.rand(num_filter_new)
print(offset)
print(filter_convolution.shape)
array_shape = array.shape
filter_shape = filter_convolution.shape

[0.59641424 0.42442125 0.46302074 0.95165906 0.27260962 0.85000078
 0.22879956 0.83222946 0.63712579 0.39152218 0.88167221 0.35683683
 0.53198938 0.21811308 0.87212711 0.0788774 ]
(16, 1, 3, 3)


In [44]:
array_shape = array.shape
print(array_shape)

(1, 30, 30)


In [45]:
start_time = time.time()
convolution_matrix = create_convolution_matrix(array.shape, 
                                               filter_convolution,
                                              offset)
end_time = time.time()
print(end_time - start_time)

2.058684825897217


In [46]:
print(convolution_matrix.shape)

(12544, 901)


In [47]:
array_extended_by_one = np.concatenate([array.flatten(), np.array([1])])

In [48]:
start_time = time.time()
a = convolution_forward(array=array_extended_by_one, conv_matrix=convolution_matrix)
end_time = time.time()
print(end_time - start_time)

1.5814740657806396


### Backward

In [49]:
@jit
def convolution_backward(convolution_matrix: np.ndarray) -> np.ndarray:
    """

    """
    return convolution_matrix[:,:-1]

In [50]:
start_time = time.time()
a = convolution_backward(convolution_matrix=convolution_matrix)
end_time = time.time()
print(end_time - start_time)

0.23185157775878906


### Update convolution filters

In [51]:
@jit
def convolution_backward_update(array: np.ndarray,
                        filter_conv_shape: Tuple[int] = (3, 3)) -> np.ndarray:
    """

    """

    array_shape = array.shape
    filter_conv_update_matrix = np.zeros(((array_shape[0] - filter_conv_shape[0] + 1) \
                                   * (array_shape[1] - filter_conv_shape[1] + 1) \
                                , filter_conv_shape[0] * filter_conv_shape[1]))
    index = 0
    for i in range(array_shape[0] - filter_conv_shape[0] + 1):
        for j in range(array_shape[1] - filter_conv_shape[1] + 1):
            filter_conv_update_matrix[index] = array[i:(i+3), j:(j+3)].flatten()
            index += 1
    

    return filter_conv_update_matrix

In [52]:
@jit
def backpropagation_update_matrix(array: np.ndarray,
                                 num_filters_old: int,
                                 num_filters_new: int,
                                 array_conv_shape: int) -> np.ndarray:
    """

    """
    filter_conv_update_matrix = np.zeros((array_conv_shape * array_conv_shape * num_filters_new, \
                                         (filter_col * filter_row) * num_filters_old * num_filters_new + num_filters_new))
    index = 0
    for k in range(num_filters_old):
        array_sliced = array[k, :,:]
        local_update_matrix = convolution_backward_update(array=array_sliced)
        for l in range(num_filters_new):
            filter_conv_update_matrix[l * array_conv_shape*array_conv_shape:(l+1) * array_conv_shape * array_conv_shape,\
            (num_filters_new * k + l) * filter_col * filter_row\
            :(num_filters_new * k + l + 1) * filter_col * filter_row] \
            = local_update_matrix

    for k in range(num_filters_new-1):
        filter_conv_update_matrix[k * array_conv_shape * array_conv_shape:(k + 1) * array_conv_shape * array_conv_shape\
        ,-(num_filters_new-k):-(num_filters_new-k-1)] = 1

    k = num_filters_new - 1
    filter_conv_update_matrix[k * array_conv_shape * array_conv_shape:(k + 1) * array_conv_shape * array_conv_shape\
        ,-1:] = 1
    
    return 1 / num_filters_old * filter_conv_update_matrix

In [53]:
num_filters_old = 16
num_filters_new = 8
filter_row, filter_col = [3, 3]
array_shape = (4, 4)
array_conv_shape = array_shape[0] - 3 + 1
array = np.random.randint(0, 255, (num_filters_old, array_shape[0], array_shape[1]))
filter_convolution = np.random.randn(num_filters_new, num_filters_old, filter_row, filter_col)
offset = np.random.rand(num_filters_new, 1).flatten()


In [54]:
start_time = time.time()
update_matrix = backpropagation_update_matrix(array=array,
                             num_filters_old=num_filters_old,
                             num_filters_new=num_filters_new,
                             array_conv_shape=array_conv_shape)
end_time = time.time()
print(end_time - start_time)

1.3108851909637451


## Initial image normalization

In [55]:
@jit
def normalize_input_image(array: np.ndarray) -> np.ndarray:
    """
    We normalize the input image tensor by dividing by 255.
    """
    return 1 / 255 * array 

## Full network

### Architecture

In [65]:
input_size = (2, 1, 30, 30) 
num_filters = [1, 16, 8]
filter_conv_row, filter_conv_col = (3, 3)


num_iterations = 1
learning_rate = 0.1
lambda_reg = 0.01
num_samples = 1

In [66]:
y_label = np.transpose(np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
]))
#print(y_label)

In [67]:
input_dimension_mlp = int(((input_size[2] - 3 + 1) / 2 - 3 + 1) **2 / 4 * num_filters[-1])
print(input_dimension_mlp)

288


In [208]:
array = np.random.randint(0, 255, input_size)
array = normalize_input_image(array=array)

### First convolution module

In [178]:
array = array[0]

In [179]:
stage = 0
array_conv_one_shape = array.shape[1] - 3 + 1
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]
filter_convolution_one = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_one = np.random.rand(num_filters_new, 1).flatten()

In [180]:
start_time = time.time()
convolution_matrix_one = create_convolution_matrix(array.shape, 
                                               filter_convolution_one,
                                              offset_one)
end_time = time.time()
print(end_time - start_time)

0.15284299850463867


In [181]:
start_time = time.time()
array_extended_by_one = np.concatenate([array.flatten(), np.array([1])])
array_conv_one = 1/num_filter_old * convolution_forward(array=array_extended_by_one, conv_matrix=convolution_matrix_one)
end_time = time.time()
print(end_time - start_time)

0.055594444274902344


In [182]:
convolution_one_backpropagation_update = backpropagation_update_matrix(array=array,
                             num_filters_old=num_filters_old,
                             num_filters_new=num_filters_new,
                             array_conv_shape=array_conv_one_shape)

In [183]:
convolution_one_backpropagation_update.shape

(12544, 160)

In [184]:
array_conv_one = array_conv_one.reshape((num_filters_new, array_conv_one_shape, array_conv_one_shape))

In [185]:
start_time = time.time()
max_pooling_one_indices, max_pooling_one = max_pooling_forward(array=array_conv_one)
end_time = time.time()
print(end_time - start_time)

0.0004267692565917969


In [186]:
start_time = time.time()
max_pooling_one_backward = max_pooling_backward(array_shape=array_conv_one.shape, array_max_indices=max_pooling_one_indices)
end_time = time.time()
print(end_time - start_time)

0.19289326667785645


In [187]:
max_pooling_one_backward.shape

(3136, 12544)

In [188]:
max_pooling_one.shape

(16, 14, 14)

In [189]:
start_time = time.time()
batch_normalization_one = batch_normalization_forward(array=max_pooling_one.flatten())
end_time = time.time()
print(end_time - start_time)

0.001712799072265625


In [190]:
batch_normalization_one

array([-0.64130684, -0.65681933,  0.71633762, ..., -0.56908925,
       -0.25821053, -0.7853155 ])

In [191]:
start_time = time.time()
batch_normalization_one_backpropagation = batch_normalization_backward(array=batch_normalization_one)
end_time = time.time()
print(end_time - start_time)

0.1548752784729004


In [192]:
batch_normalization_one_backpropagation.shape

(3136, 3136)

In [193]:
array_two = batch_normalization_one.reshape(max_pooling_one.shape)

In [194]:
array_two.shape

(16, 14, 14)

### Second convolution module

In [195]:
stage = 1
array_conv_two_shape = array_two.shape[1] - 3 + 1
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]
filter_convolution_two = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_two = np.random.rand(num_filters_new, 1).flatten()

In [52]:
start_time = time.time()
convolution_matrix_two = create_convolution_matrix(array_two.shape, 
                                               filter_convolution_two,
                                              offset_two)
end_time = time.time()
print(end_time - start_time)

0.05913233757019043


In [53]:
convolution_matrix_two.shape

(1152, 3137)

In [54]:
start_time = time.time()
array_two_extended_by_one = np.concatenate([array_two.flatten(), np.array([1])])
array_conv_two = convolution_forward(array=array_two_extended_by_one, conv_matrix=convolution_matrix_two)
end_time = time.time()
print(end_time - start_time)

0.056105852127075195


In [55]:
convolution_two_backpropagation_update = backpropagation_update_matrix(array=array_two,
                             num_filters_old=num_filters_old,
                             num_filters_new=num_filters_new,
                             array_conv_shape=array_conv_two_shape)

In [56]:
convolution_two_backpropagation_update.shape

(1152, 1160)

In [57]:
convolution_two_backward = 1/num_filter_old * convolution_backward(convolution_matrix=convolution_matrix_two)

In [58]:
convolution_two_backward.shape

(1152, 3136)

In [59]:
array_conv_two = array_conv_two.reshape((num_filters_new, array_conv_two_shape, array_conv_two_shape))

In [60]:
start_time = time.time()
max_pooling_two_indices, max_pooling_two = max_pooling_forward(array=array_conv_two)
end_time = time.time()
print(end_time - start_time)

0.00021314620971679688


In [61]:
start_time = time.time()
max_pooling_two_backward = max_pooling_backward(array_shape=array_conv_two.shape, array_max_indices=max_pooling_two_indices)
end_time = time.time()
print(end_time - start_time)

0.0006783008575439453


In [62]:
start_time = time.time()
batch_normalization_two = batch_normalization_forward(array=max_pooling_two.flatten())
end_time = time.time()
print(end_time - start_time)

0.002460956573486328


In [63]:
start_time = time.time()
batch_normalization_two_backpropagation = batch_normalization_backward(array=batch_normalization_two)
end_time = time.time()
print(end_time - start_time)

0.0006890296936035156


In [64]:
array_three = batch_normalization_two.flatten()

In [65]:
batch_normalization_two.shape

(288,)

### MLP

In [103]:
@jit
def extend_input_by_one_vector(x_input: np.ndarray) -> np.ndarray:
    
    """

    """
    height, width = x_input.shape
    vector_extended = np.ones((height + 1, width))
    vector_extended[:height, :width] = x_input
    return vector_extended

@jit
def apply_sigmoid(x_input: np.ndarray) -> np.ndarray:
    """

    """
    return 1 / (1 + np.exp(-x_input))


@jit
def forward_pass(x_input: np.ndarray,
                weight_matrix_input_layer: np.ndarray,
                weight_matrix_output_layer: np.ndarray) -> Tuple[np.ndarray]:

    """

    """

    a1 = np.dot(weight_matrix_input_layer, x_input)
    h1 = apply_sigmoid(a1)
    h1 = extend_input_by_one_vector(h1)
    a2 = np.dot(weight_matrix_output_layer, h1)
    h2 = apply_sigmoid(a2)

    return (a1, h1, a2, h2)

In [67]:
array_three = np.expand_dims(array_three, axis=0)

In [ ]:
input_dimension = array_three.shape[1]

In [68]:
hidden_layer = [8]
output_dimension = 10

In [69]:
weight_matrix_input_layer = np.random.randn(hidden_layer[0], input_dimension + 1)
weight_matrix_output_layer = np.random.randn(output_dimension, hidden_layer[0] + 1)

In [ ]:
x_input = np.transpose(array_three)
x_input = extend_input_by_one_vector(x_input=x_input)

In [70]:
a1, h1, a2, h2 = forward_pass(x_input=x_input,
                                          weight_matrix_input_layer=weight_matrix_input_layer,
                                          weight_matrix_output_layer=weight_matrix_output_layer)

update_weights_two = 0
update_weights_one = 0
i = 0
backward_one = h2[:,i] - y_label[:,i]
update_weights_two += np.stack([backward_one[j] * h1[:,i] for j in range(output_dimension)], axis=0).flatten()

update_weights_two = -1/1 * update_weights_two

backward_two = np.dot(np.transpose(backward_one), weight_matrix_output_layer[:,:-1])
backward_three = backward_two * (h1[:-1,i] * (1 - h1[:-1,i]))
update_weights_one += np.stack([backward_three[j] * x_input[:,i] for j in range(hidden_layer[0])], axis=0).flatten()

update_weights_one = -1/1 * update_weights_one

backward_four = np.dot(np.transpose(backward_three), weight_matrix_input_layer[:,:-1])
weight_matrix_output_layer = (weight_matrix_output_layer.flatten() \
                                  + learning_rate * update_weights_two).reshape(weight_matrix_output_layer.shape)
weight_matrix_input_layer = (weight_matrix_input_layer.flatten() \
                                  + learning_rate * update_weights_one).reshape(weight_matrix_input_layer.shape)

In [71]:
bp1 = np.dot(backward_four, batch_normalization_two_backpropagation)
bp2 = np.dot(bp1, max_pooling_two_backward)
bp3 = np.dot(bp2, convolution_two_backward)
update_second_conv_weights = -1/1 * np.dot(bp2, convolution_two_backpropagation_update)
update_second_conv_weights_reshaped = update_second_conv_weights[:-num_filters[2]].reshape(num_filters[2], num_filters[1], 3, 3)
filter_convolution_two = filter_convolution_two + learning_rate * update_second_conv_weights_reshaped
offset_two = offset_two + learning_rate * update_second_conv_weights[-num_filters[2]:]
bp4 = np.dot(bp3, batch_normalization_one_backpropagation)
bp5 = np.dot(bp4, max_pooling_one_backward)
update_first_conv_weights = -1/1 * np.dot(bp5, convolution_one_backpropagation_update)
update_first_conv_weights_reshaped = update_first_conv_weights[:-num_filters[1]].reshape(num_filters[1], num_filters[0], 3, 3)
filter_convolution_one = filter_convolution_one + learning_rate * update_first_conv_weights_reshaped
offset_one = offset_one + learning_rate * update_first_conv_weights[-num_filters[1]:]

# Wrap up

In [218]:
arrays = array.copy()
array_input = arrays[b]

In [239]:
stage = 0
array_conv_one_shape = array_input.shape[1] - 3 + 1
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]
filter_convolution_one = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_one = np.random.rand(num_filters_new, 1).flatten()

stage = 1
array_conv_two_shape = array_two.shape[1] - 3 + 1
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage+1]
filter_convolution_two = np.random.randn(num_filters_new, num_filters_old, filter_conv_row, filter_conv_col)
offset_two = np.random.rand(num_filters_new, 1).flatten()

hidden_layer = [8]
output_dimension = 10
weight_matrix_input_layer = np.random.randn(hidden_layer[0], input_dimension_mlp + 1)
weight_matrix_output_layer = np.random.randn(output_dimension, hidden_layer[0] + 1)

In [220]:
#@jit
def train(num_epochs: int,
         num_filters: list,
         filter_convolution_one: np.ndarray,
         filter_convolution_two: np.ndarray,
         offset_one: np.ndarray,
         offset_two: np.ndarray,
         arrays: np.ndarray,
         weight_matrix_input_layer: np.ndarray,
         weight_matrix_output_layer: np.ndarray) -> np.ndarray:
    
    batch_size = arrays.shape[0]
    for k in range(num_epochs):
        start_time = time.time()
        update_weights_two = 0
        update_weights_one = 0
        update_second_conv_weights = 0
        update_first_conv_weights = 0
        
        for b in range(batch_size):
            array = arrays[b]
            stage = 0
            array_conv_one_shape = array.shape[1] - 3 + 1
            num_filters_old = num_filters[stage]
            num_filters_new = num_filters[stage + 1]
            convolution_matrix_one = create_convolution_matrix(array.shape, 
                                                           filter_convolution_one,
                                                          offset_one)

            
            array_extended_by_one = np.concatenate([array.flatten(), np.array([1])])
            
            array_conv_one = 1/num_filter_old * convolution_forward(array=array_extended_by_one, conv_matrix=convolution_matrix_one)
            
            convolution_one_backpropagation_update = backpropagation_update_matrix(array=array,
                                         num_filters_old=num_filters_old,
                                         num_filters_new=num_filters_new,
                                         array_conv_shape=array_conv_one_shape)
            
            array_conv_one = array_conv_one.reshape((num_filters_new, array_conv_one_shape, array_conv_one_shape))
            
            max_pooling_one_indices, max_pooling_one = max_pooling_forward(array=array_conv_one)
            max_pooling_one_backward = max_pooling_backward(array_shape=array_conv_one.shape, array_max_indices=max_pooling_one_indices)
            
            batch_normalization_one = batch_normalization_forward(array=max_pooling_one.flatten())
            batch_normalization_one_backpropagation = batch_normalization_backward(array=batch_normalization_one)
            
            array_two = batch_normalization_one.reshape(max_pooling_one.shape)
            
            stage = 1
            array_conv_two_shape = array_two.shape[1] - 3 + 1
            num_filters_old = num_filters[stage]
            num_filters_new = num_filters[stage + 1]
            convolution_matrix_two = create_convolution_matrix(array_two.shape, 
                                                           filter_convolution_two,
                                                          offset_two)
            convolution_two_backward = 1/num_filter_old * convolution_backward(convolution_matrix=convolution_matrix_two)
    
    
            
            
            array_two_extended_by_one = np.concatenate([array_two.flatten(), np.array([1])])
            
            
            array_conv_two = convolution_forward(array=array_two_extended_by_one, conv_matrix=convolution_matrix_two)
            convolution_two_backpropagation_update = backpropagation_update_matrix(array=array_two,
                                         num_filters_old=num_filters_old,
                                         num_filters_new=num_filters_new,
                                         array_conv_shape=array_conv_two_shape)
            
            array_conv_two = array_conv_two.reshape((num_filters_new, array_conv_two_shape, array_conv_two_shape))
            
            max_pooling_two_indices, max_pooling_two = max_pooling_forward(array=array_conv_two)
            max_pooling_two_backward = max_pooling_backward(array_shape=array_conv_two.shape, array_max_indices=max_pooling_two_indices)
            
            batch_normalization_two = batch_normalization_forward(array=max_pooling_two.flatten())
            batch_normalization_two_backpropagation = batch_normalization_backward(array=batch_normalization_two)
            
            array_three = batch_normalization_two.flatten()
            array_three = np.expand_dims(array_three, axis=0)
            
            x_input = np.transpose(array_three)
            x_input = extend_input_by_one_vector(x_input=x_input)
            
            a1, h1, a2, h2 = forward_pass(x_input=x_input,
                                                      weight_matrix_input_layer=weight_matrix_input_layer,
                                                      weight_matrix_output_layer=weight_matrix_output_layer)
            

            backward_one = (h2[:,0] - y_label[:,b])
            update_weights_two += np.stack([backward_one[j] * h1[:,0] for j in range(output_dimension)], axis=0).flatten()
            
            #update_weights_two = -1/batch_size * update_weights_two
            
            backward_two = np.dot(np.transpose(backward_one), weight_matrix_output_layer[:,:-1])
            backward_three = backward_two * (h1[:-1,0] * (1 - h1[:-1,0]))
            update_weights_one += np.stack([backward_three[j] * x_input[:,0] for j in range(hidden_layer[0])], axis=0).flatten()
            
            #update_weights_one = -1/batch_size * update_weights_one
            
            backward_four = np.dot(np.transpose(backward_three), weight_matrix_input_layer[:,:-1])
            #weight_matrix_output_layer = (weight_matrix_output_layer.flatten() \
                                              #+ learning_rate * update_weights_two).reshape(weight_matrix_output_layer.shape)
            #weight_matrix_input_layer = (weight_matrix_input_layer.flatten() \
                                              #+ learning_rate * update_weights_one).reshape(weight_matrix_input_layer.shape)
            
            bp1 = np.dot(backward_four, batch_normalization_two_backpropagation)
            bp2 = np.dot(bp1, max_pooling_two_backward)
            bp3 = np.dot(bp2, convolution_two_backward)
            update_second_conv_weights += np.dot(bp2, convolution_two_backpropagation_update)
            #update_second_conv_weights_reshaped = update_second_conv_weights[:-num_filters[2]].reshape(num_filters[2], num_filters[1], 3, 3)
            #filter_convolution_two = filter_convolution_two + learning_rate * update_second_conv_weights_reshaped
            #offset_two = offset_two + learning_rate * update_second_conv_weights[-num_filters[2]:]
            bp4 = np.dot(bp3, batch_normalization_one_backpropagation)
            bp5 = np.dot(bp4, max_pooling_one_backward)
            update_first_conv_weights += np.dot(bp5, convolution_one_backpropagation_update)
            #update_first_conv_weights_reshaped = update_first_conv_weights[:-num_filters[1]].reshape(num_filters[1], num_filters[0], 3, 3)
            #filter_convolution_one = filter_convolution_one + learning_rate * update_first_conv_weights_reshaped
            #offset_one = offset_one + learning_rate * update_first_conv_weights[-num_filters[1]:]
            
        end_time = time.time()
        print(f"Processing time for one epoch: {end_time - start_time}")
        #print(f"Probability: {h2[0,0]}")

        
        update_weights_two = -1/batch_size * update_weights_two

        update_weights_one = -1/batch_size * update_weights_one

        weight_matrix_output_layer = (weight_matrix_output_layer.flatten() \
                                      + learning_rate * update_weights_two).reshape(weight_matrix_output_layer.shape)
        weight_matrix_input_layer = (weight_matrix_input_layer.flatten() \
                                      + learning_rate * update_weights_one).reshape(weight_matrix_input_layer.shape)

        update_second_conv_weights = -1/batch_size * update_second_conv_weights
        update_second_conv_weights_reshaped = update_second_conv_weights[:-num_filters[2]].reshape(num_filters[2], num_filters[1], 3, 3)
        filter_convolution_two = filter_convolution_two + learning_rate * update_second_conv_weights_reshaped
        offset_two = offset_two + learning_rate * update_second_conv_weights[-num_filters[2]:]

        update_first_conv_weights = -1/batch_size * update_first_conv_weights
        update_first_conv_weights_reshaped = update_first_conv_weights[:-num_filters[1]].reshape(num_filters[1], num_filters[0], 3, 3)
        filter_convolution_one = filter_convolution_one + learning_rate * update_first_conv_weights_reshaped
        offset_one = offset_one + learning_rate * update_first_conv_weights[-num_filters[1]:]
    
    return weight_matrix_output_layer, weight_matrix_input_layer, filter_convolution_two, offset_two, filter_convolution_one, offset_one

In [221]:
#print(y_label)

In [252]:
weight_matrix_output_layer, weight_matrix_input_layer, filter_convolution_two, offset_two, filter_convolution_one, offset_one = train(num_epochs=10,
      num_filters=num_filters,
         filter_convolution_one=filter_convolution_one,
         filter_convolution_two=filter_convolution_two,
         offset_one=offset_one,
         offset_two=offset_two,
         arrays=arrays,
         weight_matrix_input_layer=weight_matrix_input_layer,
         weight_matrix_output_layer=weight_matrix_output_layer)

Processing time for one epoch: 1.3677456378936768
Processing time for one epoch: 1.3158118724822998
Processing time for one epoch: 1.63834547996521
Processing time for one epoch: 1.3010611534118652
Processing time for one epoch: 1.7076952457427979
Processing time for one epoch: 1.627326488494873
Processing time for one epoch: 1.772806167602539
Processing time for one epoch: 1.5833828449249268
Processing time for one epoch: 1.7358715534210205
Processing time for one epoch: 1.9203119277954102


In [253]:
b = 1

In [254]:
array_input = arrays[b]
stage = 0
array_conv_one_shape = array_input.shape[1] - 3 + 1
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage + 1]
convolution_matrix_one = create_convolution_matrix(array_input.shape, 
                                               filter_convolution_one,
                                              offset_one)
array_extended_by_one = np.concatenate([array_input.flatten(), np.array([1])])
array_conv_one = 1/num_filter_old * convolution_forward(array=array_extended_by_one, conv_matrix=convolution_matrix_one)
array_conv_one = array_conv_one.reshape((num_filters_new, array_conv_one_shape, array_conv_one_shape))
max_pooling_one_indices, max_pooling_one = max_pooling_forward(array=array_conv_one)
batch_normalization_one = batch_normalization_forward(array=max_pooling_one.flatten())
array_two = batch_normalization_one.reshape(max_pooling_one.shape)
stage = 1
array_conv_two_shape = array_two.shape[1] - 3 + 1
num_filters_old = num_filters[stage]
num_filters_new = num_filters[stage + 1]
convolution_matrix_two = create_convolution_matrix(array_two.shape, 
                                               filter_convolution_two,
                                              offset_two)
array_two_extended_by_one = np.concatenate([array_two.flatten(), np.array([1])])
array_conv_two = convolution_forward(array=array_two_extended_by_one, conv_matrix=convolution_matrix_two)
array_conv_two = array_conv_two.reshape((num_filters_new, array_conv_two_shape, array_conv_two_shape))
max_pooling_two_indices, max_pooling_two = max_pooling_forward(array=array_conv_two)
batch_normalization_two = batch_normalization_forward(array=max_pooling_two.flatten())
array_three = batch_normalization_two.flatten()
array_three = np.expand_dims(array_three, axis=0)
x_input = np.transpose(array_three)
x_input = extend_input_by_one_vector(x_input=x_input)
a1, h1, a2, h2 = forward_pass(x_input=x_input,
                                          weight_matrix_input_layer=weight_matrix_input_layer,
                                          weight_matrix_output_layer=weight_matrix_output_layer)

In [255]:
h2

array([[0.47280402],
       [0.10461708],
       [0.52024437],
       [0.04577035],
       [0.13230358],
       [0.10674013],
       [0.10325253],
       [0.08217306],
       [0.05620791],
       [0.09387318]])

In [88]:
#array_two_flatten = array_two.flatten()
    
#array_two_extended_by_one = np.zeros(array_two_flatten.size + 1)
    
#array_two_extended_by_one[:array_two_flatten.size] = array_two_flatten